<a href="https://colab.research.google.com/github/davidandw190/pytorch-deep-learning-workspace/blob/main/ml-ops/dc-fares/00_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ML-Ops | DC Taxi Fares**

## 00 - Data Preprocessing



### Downloading the DC taxi dataset

`gdown` is a Python utility for downloading files stored in Google Drive. The bash script in the following cell iterates through a collection of Google Drive identifiers that match files `taxi_2015.zip` through `taxi_2019.zip` stored in a shared Google Drive. This script uses these files instead of the original files from https://opendata.dc.gov/search?categories=transportation&q=taxi&type=document%20link since the originals cannot be easily downloaded using a bash script.

In [ ]:
%%bash
pip install gdown
for ID in '1yF2hYrVjAZ3VPFo1dDkN80wUV2eEq65O'\
          '1Z7ZVi79wKEbnc0FH3o0XKHGUS8MQOU6R'\
          '1I_uraLKNbGPe3IeE7FUa9gPfwBHjthu4'\
          '1MoY3THytktrxam6_hFSC8kW1DeHZ8_g1'\
          '1balxhT6Qh_WDp4wq4OsG40iwqFa86QgW'
do

  gdown --id $ID

done

### Unziping the dataset

The script in the following cell unzips the downloaded dataset files to the `dctaxi` subdirectory in the current directory of the notebook.

The `-o` flag used by the `unzip` command overwrites existing files in case we execute the next cell more than once.


In [ ]:
%%bash

mkdir -p dctaxi

for YEAR in '2015' \
            '2016' \
            '2017' \
            '2018' \
            '2019'
do

  unzip -o taxi_$YEAR.zip -d dctaxi

done

### Reporting on the disk space used by the dataset files

The next cell reports on the disk usage (`du`) by the files from the DC taxi dataset. All of the files in the dataset have the `taxi_` prefix.

Since the entire output of the `du` command lists the disk usage of all of the files, the `tail` command is used to limit the output to just the last line. We can remove the tail command we wish to report on the disk usage by the individual files in the dataset.



In [3]:
!du -cha --block-size=1MB dctaxi/taxi_*.txt | tail -n 1

11987	total


### Scanning the dataset documentation

The dataset includes a `README_DC_Taxicab_trip.txt` file with a brief documentation about the dataset contents. We will run the next cell and take a moment to review the documentation, focusing on the schema used by the dataset.



In [5]:
%%bash
cat dctaxi/README_DC_Taxicab_trip.txt

TITLE: Taxicab Trip Information

ABSTRACT:  DC Taxi trips from January 2015 to June 2019.  The zip file contains zip files of pipe (|) delimeted text files for trips by month.  The record counts are:

Month			Count
5/2015		1,397,102 
6/2015		1,470,466 
7/2015		1,401,792 
8/2015		1,129,707 
9/2015		1,308,445 
10/2015		1,487,133 
11/2015	 	  993,502 
12/2015		1,081,726 
1/2016	 	  922,338 
2/2016		1,194,698 
3/2016		1,404,639 
4/2016		1,369,882 
5/2016		1,323,155 
6/2016		1,282,651 
7/2016		1,109,118 
8/2016		  949,650 
9/2016		1,203,388 
10/2016		1,027,036 
11/2016		1,011,673 
12/2016	 	  898,993 
1/2017	  	  901,807 
2/2017	 	  949,578 
3/2017		1,237,621
4/2017		1,185,859
5/2017		1,188,944
6/2017		1,182,526
7/2017		  993,834
8/2017     	  813,513
9/2017		  902,795
10/2017    	1,056,864
11/2017    	  867,709
12/2017    	  730,679
1/2018     	  686,993
2/2018     	  750,363
3/2018     	  999,048
4/2018     	1,040,147
5/2018     	  984,593
6/2018     	  941,608
7/2018     	  810,292
8/201

### Previewing the dataset

We run the next cell to confirm that the dataset consists of pipe (|) separated values organized according to the schema described by the documentation. The `taxi_2015_09.txt` file used in the next cell was picked arbitrarily, just to illustrate the dataset.

In [6]:
!head dctaxi/taxi_2015_09.txt

OBJECTID|TRIPTYPE|PROVIDERNAME|FAREAMOUNT|GRATUITYAMOUNT|SURCHARGEAMOUNT|EXTRAFAREAMOUNT|TOLLAMOUNT|TOTALAMOUNT|PAYMENTTYPE|ORIGINCITY|ORIGINSTATE|ORIGINZIP|DESTINATIONCITY|DESTINATIONSTATE|DESTINATIONZIP|MILEAGE|DURATION|ORIGIN_BLOCK_LATITUDE|ORIGIN_BLOCK_LONGITUDE|ORIGIN_BLOCKNAME|DESTINATION_BLOCK_LATITUDE|DESTINATION_BLOCK_LONGITUDE|DESTINATION_BLOCKNAME|AIRPORT|ORIGINDATETIME_TR|DESTINATIONDATETIME_TR
6482604|1|Transco|3.25|0.00|0.25|0.00|0.00|3.50|1|Washington|DC|20002|Washington|DC|20002|0|1|38.897204|-77.008388|1 - 99 BLOCK OF MASSACHUSETTS AVENUE NE|38.896039|-77.007142|40 - 49 BLOCK OF COLUMBUS CIRCLE NE|N|09/01/2015 00:00|09/01/2015 00:00
6482605|1|Transco|7.30|1.89|0.25|0.00|0.00|9.44|1|Washington|DC|20005|Washington|DC|20009|2|7|38.904692|-77.034573|1100 - 1199 BLOCK OF 15TH STREET NW|38.921623|-77.042365|2400 - 2499 BLOCK OF 18TH STREET NW|N|09/01/2015 00:00|09/01/2015 00:00
6482606|1|Transco|5.68|0.00|0.25|0.00|0.00|5.93|2||NULL|00000||NULL|00000|1|3|||||||N|09/01/2015 0

### Downloading and install AWS CLI



In [ ]:
%%bash
curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
unzip -o awscliv2.zip
sudo ./aws/install

### Configuring AWS credentials and region
The contents of the next cell should be modified to specify AWS credentials as strings.

**This will soon be replaced with a more secure and automated way of handling secrets*

If you see the following exception:

`TypeError: str expected, not NoneType`

It means that you did not specify the credentials correctly.

In [ ]:
import os
# *** REPLACE None in the next 2 lines with your AWS key values ***
os.environ['AWS_ACCESS_KEY_ID'] = None
os.environ['AWS_SECRET_ACCESS_KEY'] = None

Run the next cell to validate your credentials.

In [ ]:
%%bash
aws sts get-caller-identity

In [ ]:
# *** REPLACE None in the next line with your AWS region ***
os.environ['AWS_DEFAULT_REGION'] = None

If you have specified the region correctly, the following cell should return back the region that you have specifies.

In [ ]:
%%bash
echo $AWS_DEFAULT_REGION

### Creating unique bucket ID